# Compare the model caption inference against the ground truth captions from annotations file.
# Trained model on 97k of some 100k images of Coco_Train2017 dataset. Balance 3k kept aside as the Validation Set. Using all the 5k images of Coco_Val2017 dataset as the Test dataset.

## Encoder: Pre-trained Google Inception-v3 trained on Imagenet imported directly from Keras
## Decoder: Trained with the 97k data and their captions from annotations file.

## AVAILABLE data :
### Coco_Train2017     = has 118287 images
### Coco_Val2017       = has 5000   images
### Combined total     = 123287     images

## USED data :
### Coco_Train2017     = 100000  images
### Coco_Val2017       = 5000    images
### Combined total     = 105000  images

### Using only the first 100k images of Train2017 + all 5k images of Val2017
### Thus total data available for training = 100k + 5k = 105k
### Details of split of data:
### Training   data = 97000 images from Coco_Train2017
### Validation data = 3000  images from Coco_Train2017
### Test       data = 5000  images from Coco_Val2017

### Note: 1) Each image will have multiple captions (up to 5 as some may be discarded)
###       2) Not using the Coco_Test2017 dataset at all as it has no annotations json file which has the captions.

## Using this LINK: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8
## Image captioning with Keras

In [1]:
import os
import sys
import json
import time
import datetime
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import re
import pickle
#import itertools
import PIL
import PIL.Image

## import numpy as np
## from numpy import array
## import pandas as pd
## import matplotlib.pyplot as plt
## %matplotlib inline
## import string
## import os
## from PIL import Image
## import glob
## from pickle import dump, load
## from time import time
## from keras.preprocessing import sequence
## from keras.models import Sequential
## from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
## from keras.optimizers import Adam, RMSprop
## from keras.layers.wrappers import Bidirectional
## from keras.layers.merge import add
## from keras.applications.inception_v3 import InceptionV3
## from keras.preprocessing import image
## from keras.models import Model
## from keras import Input, layers
## from keras import optimizers
## from keras.applications.inception_v3 import preprocess_input
## from keras.preprocessing.text import Tokenizer
## from keras.preprocessing.sequence import pad_sequences
## from keras.utils import to_categorical

In [2]:
#from google.colab import drive
#drive.flush_and_unmount()
#drive.mount('/content/gdrive')

In [3]:
## LAPTOP versions

## Weights from training till now
#OPDIR = r'../working/'

## Images locations
#IPDIR_IMGS_COCO_TRAIN = r'../input/coco-2017-dataset/coco2017/train2017/'
IPDIR_IMGS_COCO_VAL = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_val2017_5k/val2017/'

## Annotations json file location from where to pick up the captions
IPDIR_ANNO = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/coco_2017_annotations/'

## Bottleneck CNN Encoder output for all the images of TRAIN , VALIDATION and TEST DATASETS
#IPDIR_IMG_ENCODINGS = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_ImgEncodings_1/'

## to make deterministic - saved all the variables based on the data used to train during phase 1
## ALL SUBSEQUENT PHASES WILL RELOAD FROM THE DATA IN THIS LOCATION
IPDIR_DETERMINISTIC = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Run2_Deterministic_Info/'

## Weights from previous stage of training
IPDIR_WEIGHTS_IN = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Weights_In_Run2/'

"""
## Kaggle versions

## Weights from training till now
OPDIR = r'../working/'

## Images locations
IPDIR_IMGS_COCO_TRAIN = r'../input/coco-2017-dataset/coco2017/train2017/'
IPDIR_IMGS_COCO_VAL = r'../input/coco-2017-dataset/coco2017/val2017/'

## Annotations json file location from where to pick up the captions
IPDIR_ANNO = r'../input/coco-2017-dataset/coco2017/annotations/'

## Bottleneck CNN Encoder output for all the images to be used in training
IPDIR_IMG_ENCODINGS = r'../input/thesis-imgcap-imgencodings-1/'

## to make deterministic - saved all the variables based on the data used to train during phase 1
## ALL SUBSEQUENT PHASES WILL RELOAD FROM THE DATA IN THIS LOCATION
IPDIR_DETERMINISTIC = '../input/thesis-imgcap-run2-deterministic-info/'

## Weights from previous stage of training
IPDIR_WEIGHTS_IN = r'../input/imgcap-kagg-run2-weights-in/'


#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/' #not needed as already created the embedding matrix and pickled

## Google drive versions
#OPDIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/ImgCapTraining/CocoVal2017_1/'
#IPDIRIMGS = '/content/gdrive/My Drive/ThesisStoryGen/Data/coco_val2017_wget_1/val2017/'
#IPDIRANNO = '/content/gdrive/My Drive/ThesisStoryGen/Data/coco_annotations_trainval2017/'
#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/'

"""

"\n## Kaggle versions\n\n## Weights from training till now\nOPDIR = r'../working/'\n\n## Images locations\nIPDIR_IMGS_COCO_TRAIN = r'../input/coco-2017-dataset/coco2017/train2017/'\nIPDIR_IMGS_COCO_VAL = r'../input/coco-2017-dataset/coco2017/val2017/'\n\n## Annotations json file location from where to pick up the captions\nIPDIR_ANNO = r'../input/coco-2017-dataset/coco2017/annotations/'\n\n## Bottleneck CNN Encoder output for all the images to be used in training\nIPDIR_IMG_ENCODINGS = r'../input/thesis-imgcap-imgencodings-1/'\n\n## to make deterministic - saved all the variables based on the data used to train during phase 1\n## ALL SUBSEQUENT PHASES WILL RELOAD FROM THE DATA IN THIS LOCATION\nIPDIR_DETERMINISTIC = '../input/thesis-imgcap-run2-deterministic-info/'\n\n## Weights from previous stage of training\nIPDIR_WEIGHTS_IN = r'../input/imgcap-kagg-run2-weights-in/'\n\n\n#WORD_EMBEDDINGS_DIR = '/content/gdrive/My Drive/ThesisStoryGen/Data/WordEmbeddings/' #not needed as already cre

# Reload everything needed from pickled files created based on a certain split of Train and Validation datasets 97k:3k.
## Run necessary code again to check all good to proceed with Training phase:
## Outcome expected:
## 1) unique words in original vocabulary = 24323
## 2) unique words in culled vocab vocab_threshold = 6757
##    Thus, VOCAB_SIZE = 6757 + 1 = 6758
## 3) Embedding matrix shape should be (6758,200)

In [4]:
IPDIR_DETERMINISTIC

'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Run2_Deterministic_Info/'

In [5]:
os.listdir(IPDIR_DETERMINISTIC)

['descriptions_test.pkl',
 'descriptions_train_97000.pkl',
 'descriptions_train_97000_startend.pkl',
 'descriptions_train_and_val.pkl',
 'descriptions_val_3000.pkl',
 'embedding_matrix_6758_200.pkl',
 'img_encodings_train_97000.pkl',
 'img_encodings_val_3000.pkl',
 'ixtoword_train_97000.pkl',
 'train_imgs_keys_97000_randomSplit444.pkl',
 'val_imgs_keys_3000_randomSplit444.pkl',
 'wordtoix_train_97000.pkl']

In [6]:
## Reload all the files to make it deterministic


# variable = img_encodings_train
with open(IPDIR_DETERMINISTIC + 'img_encodings_train_97000.pkl', 'rb') as handle:
    img_encodings_train = pickle.load(handle)

# variable = img_encodings_val
with open(IPDIR_DETERMINISTIC + 'img_encodings_val_3000.pkl', 'rb') as handle:
    img_encodings_val = pickle.load(handle)

# variable = descriptions_train
with open(IPDIR_DETERMINISTIC + 'descriptions_train_97000_startend.pkl', 'rb') as handle:
    descriptions_train = pickle.load(handle)

# variable = descriptions_val
with open(IPDIR_DETERMINISTIC + 'descriptions_val_3000.pkl', 'rb') as handle:
    descriptions_val = pickle.load(handle)

# variable = train_imgs_keys
with open(IPDIR_DETERMINISTIC + 'train_imgs_keys_97000_randomSplit444.pkl', 'rb') as handle:
    train_imgs_keys = pickle.load(handle)

# variable = val_imgs_keys
with open(IPDIR_DETERMINISTIC + 'val_imgs_keys_3000_randomSplit444.pkl', 'rb') as handle:
    val_imgs_keys = pickle.load(handle)

# variable = embedding_matrix
with open(IPDIR_DETERMINISTIC + 'embedding_matrix_6758_200.pkl', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

# variable = ixtoword
with open(IPDIR_DETERMINISTIC + 'ixtoword_train_97000.pkl', 'rb') as handle:
    ixtoword = pickle.load(handle)

# variable = wordtoix
with open(IPDIR_DETERMINISTIC + 'wordtoix_train_97000.pkl', 'rb') as handle:
    wordtoix = pickle.load(handle)

print(f"Reloaded it all")

Reloaded it all


In [7]:
## check the reloaded files

In [8]:
print(f"Encodings data:\nlen(img_encodings_train) = {len(img_encodings_train)}\t\tlen(img_encodings_val) = {len(img_encodings_val)}")
print(f"Descriptions data:\nlen(descriptions_train) = {len(descriptions_train)}\t\tlen(descriptions_val) = {len(descriptions_val)}")
print(f"\nCHECK : reloaded values = 97k for Train , 3k for Validation")

Encodings data:
len(img_encodings_train) = 97000		len(img_encodings_val) = 3000
Descriptions data:
len(descriptions_train) = 97000		len(descriptions_val) = 3000

CHECK : reloaded values = 97k for Train , 3k for Validation


In [9]:
## at this stage the descriptions_train already has the start and end tokens added to it
vocabulary = set()
for key in descriptions_train.keys():
    [vocabulary.update(d.split()) for d in descriptions_train[key]]
print(f"Original Vocabulary Size with all words = {len(vocabulary)}")
print(f"\nCHECK : reloaded value = 24323")

Original Vocabulary Size with all words = 24323

CHECK : reloaded value = 24323


In [10]:
# Create a list of all the training captions, find the freq and retain words where the freq > threshold chosen

all_desc_in_training_samples = []
for key, val in descriptions_train.items():
    for cap in val:
        all_desc_in_training_samples.append(cap)

MIN_WORD_COUNT_THRESHOLD = 10
word_counts = {}
nsents = 0
for each_desc in all_desc_in_training_samples:
    nsents += 1
    for w in each_desc.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab_threshold = [w for w in word_counts if word_counts[w] >= MIN_WORD_COUNT_THRESHOLD]

print(f"Culled vocabulary to only retain words occurring more than threshold = {MIN_WORD_COUNT_THRESHOLD} times.\nNew vocab size , len(vocab_threshold) = {len(vocab_threshold)}")
print(f"\nCHECK : reloaded value = 6757")

Culled vocabulary to only retain words occurring more than threshold = 10 times.
New vocab size , len(vocab_threshold) = 6757

CHECK : reloaded value = 6757


In [11]:
## determine the maximum sequence length - parameter MAX_LENGTH_CAPTION used during the RNN deocder model setup

## convert a dictionary of clean descriptions to a list of descriptions
def extract_each_desc(_descriptions):
    all_desc = list()
    for key in _descriptions.keys():
        [all_desc.append(d) for d in _descriptions[key]]
    return all_desc

## find the longest description length
def find_max_length_desc(_descriptions):
    desc_sentences = extract_each_desc(_descriptions)
    return max(len(d.split()) for d in desc_sentences)

MAX_LENGTH_CAPTION = find_max_length_desc(descriptions_train)  ## will be used directly later while defining Decoder model
print(f"Max Description Length: {MAX_LENGTH_CAPTION}")
print(f"\nCHECK : reloaded value = 49")

Max Description Length: 49

CHECK : reloaded value = 49


In [12]:
## the value now, as it will be used as:: VOCAB_SIZE = len(wordtoix) + 1
print(f"len(wordtoix) = {len(wordtoix)}")
print(f"\nCHECK : reloaded value = 6757")

VOCAB_SIZE = len(wordtoix) + 1
print(f"\n\nSet the   VOCAB_SIZE = len(wordtoix) + 1 = {VOCAB_SIZE}")

EMBEDDING_DIMS = 200
print(f"\n\nSet the   EMBEDDING_DIMS = {EMBEDDING_DIMS}")

len(wordtoix) = 6757

CHECK : reloaded value = 6757


Set the   VOCAB_SIZE = len(wordtoix) + 1 = 6758


Set the   EMBEDDING_DIMS = 200


In [13]:
## see the index output by wordtoix for the start and end sequence tokens as well as some random one word
print( wordtoix.get('startseq') , wordtoix.get('endseq') , wordtoix.get('cat') )

1 9 526


In [14]:
print(f"Shape of embedding_matrix = {embedding_matrix.shape}")
print(f"\nCHECK : reloaded shape = 6758,200")

Shape of embedding_matrix = (6758, 200)

CHECK : reloaded shape = 6758,200


In [15]:
## Recheck these parameters that will define the Decoder architecture ::: EMBEDDING_DIMS , VOCAB_SIZE , MAX_LENGTH_CAPTION
print(f"SHOULD be:\nEMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49")
print(f"\nThe values to be used in Decoder setup:\nEMBEDDING_DIMS = {EMBEDDING_DIMS} , VOCAB_SIZE = {VOCAB_SIZE} , MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")

SHOULD be:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49

The values to be used in Decoder setup:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49


## delete all the unused variable as not important for this inference and comparison processing

In [16]:
del img_encodings_train, img_encodings_val, descriptions_train, descriptions_val, train_imgs_keys, val_imgs_keys, embedding_matrix

# NOT USING ENCODINGS - WILL PASS IMAGE THRU ENCODER TO EXTRACT ENCODINGS

## Reload the Coco_Val_2017 5k image encodings from pickled file.
### NOTE: This is the TEST dataset I am using which has the ground truth captions for comparison against my models inference.

In [17]:
IPDIR_IMG_ENCODINGS

NameError: name 'IPDIR_IMG_ENCODINGS' is not defined

In [ ]:
os.listdir(IPDIR_IMG_ENCODINGS)

In [ ]:
## From Val2017 images pickled files
print(f"\n\nLoading encodings for the test images")
pickled_encodings_files_val2017 = ['val2017_all_5k_images_encoded_features_pickled_2.pkl']
img_encodings_test = {}
for idx, subset_encodings_file_val2017 in enumerate(pickled_encodings_files_val2017):
    with open(IPDIR_IMG_ENCODINGS + subset_encodings_file_val2017, 'rb') as handle:
        img_encodings_test.update(pickle.load(handle))
        print(f"Loaded file num {idx+1} :: {subset_encodings_file_val2017}")
    print(f"Number of entries in img_encodings_test = {len(img_encodings_test)}")
print(f"\nFinal count img_encodings_test = {len(img_encodings_test)}. Should be = 5k.")

del pickled_encodings_files_val2017

In [ ]:
## Thus "img_encodings_test" has the CNN encoder output feature encodings for all 5k images

## Use the annotations file to load the descriptions and clean as per standard processing followed overall.

In [18]:
IPDIR_ANNO

'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/coco_2017_annotations/'

In [19]:
os.listdir(IPDIR_ANNO)

['captions_train2017.json', 'captions_val2017.json']

In [20]:
# build the descriptions_test_dataset dict
## BUT NOTE: it is actually from the captions_val2017 dataset - which I treat as my TEST DATASET

with open(IPDIR_ANNO+'captions_val2017.json', 'r') as f:
  data = json.load(f)
  #data.keys() # dict_keys(['info', 'licenses', 'images', 'annotations'])
  #type(data['annotations']) # is a list
  #type(data['images'])      # also is a list

dfanno = pd.DataFrame(data=data['annotations'])
# dfanno.columns = Index(['image_id', 'id', 'caption'], dtype='object') 
# dfanno.dtypes =
#   image_id     int64
#   id           int64
#   caption     object
#   dtype: object

dfimages = pd.DataFrame(data=data['images'])
# dfimages.columns = Index(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'], dtype='object')
# dfimages.dtypes =
#   license           int64
#   file_name        object
#   coco_url         object
#   height            int64
#   width             int64
#   date_captured    object
#   flickr_url       object
#   id                int64
#   dtype: object
## of above, am dropping useless columns
dfimages.drop(['license', 'coco_url', 'date_captured', 'flickr_url'], axis = 1, inplace=True)

## columns remaining in the dfs are:
# dfanno columns are      image_id , id , caption
#                         179765   , 38 ,	A black Honda motorcycle parked in front of a ...
# dfimages columns are    file_name        , height ,  width , id
#                         000000397133.jpg , 427    ,  640   , 397133

## the captions are not ordered for each image and seem to be randomly placed

dfanno.head(3)

,image_id,id,caption
0,179765,38,A black Honda motorcycle parked in front of a ...
1,179765,182,A Honda motorcycle parked in a grass driveway
2,190236,401,An office cubicle with four different types of...


In [21]:
## marrying the "annotations" and the "images" keys info together into dfanno
## the file_name column has the actual image name from the "image" key section
dfanno = dfanno.merge(dfimages, how="inner", left_on='image_id', right_on='id')
dfanno.drop(['height', 'width', 'id_y'], axis = 1, inplace=True)
dfanno.rename(columns={'id_x':'id'}, inplace=True)
dfanno.head(3)

,image_id,id,caption,file_name
0,179765,38,A black Honda motorcycle parked in front of a ...,000000179765.jpg
1,179765,182,A Honda motorcycle parked in a grass driveway,000000179765.jpg
2,179765,479,A black Honda motorcycle with a dark burgundy ...,000000179765.jpg


In [22]:
## Mapping image with captions using dictionary

def create_descriptions_dictionary(_dfin):
    descriptions = {}
    for row in _dfin.itertuples():
      rowdict = row._asdict()
      img_filename = rowdict['file_name'].split('.')[0] # drop the .jpg part
      img_caption = rowdict['caption']
      if(img_filename not in descriptions):
        descriptions[img_filename] = [img_caption]
      else:
        descriptions[img_filename].append(img_caption)
    return descriptions

descriptions_test = create_descriptions_dictionary(dfanno)
print(f"len(descriptions_test) = {len(descriptions_test)}")

del dfanno

len(descriptions_test) = 5000


## Clean the description for test dataset

In [23]:
dict(list(descriptions_test.items())[:2])

{'000000179765': ['A black Honda motorcycle parked in front of a garage.',
  'A Honda motorcycle parked in a grass driveway',
  'A black Honda motorcycle with a dark burgundy seat.',
  'Ma motorcycle parked on the gravel in front of a garage',
  'A motorcycle with its brake extended standing outside'],
 '000000190236': ['An office cubicle with four different types of computers.',
  'The home office space seems to be very cluttered.',
  'an office with desk computer and chair and laptop.',
  'Office setting with a lot of computer screens.',
  'A desk and chair in an office cubicle.']}

In [24]:
## example of caption with accidental newline \n in the caption
descriptions_test['000000482917']

['A dog sitting between its masters feet on a footstool watching tv\n',
 'A dog between the feet of a person looking at a TV.',
 'A dog and a person are watching television together.',
 'A person is sitting with their dog watching tv.',
 'A man relaxing at home, watching television with his dog.']

In [25]:
# prepare translation table for removing punctuation
## string.punctuation  gives   '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' and will take care of all these characters being made into a space
tran_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
for key, desc_list in descriptions_test.items():
    for idx in range(len(desc_list)):
        desc = desc_list[idx]
        # replace all punctuation with space in description before tokenizing
        desc = desc.translate(tran_table)
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove any non-alphabetic tokens
        desc = [word for word in desc if word.isalpha()]
        # overwrite with cleaned description
        desc_list[idx] =  ' '.join(desc)

In [26]:
## example of caption with accidental newline \n in the caption  -- POST CLEANUP
descriptions_test['000000482917']

['dog sitting between its masters feet on footstool watching tv',
 'dog between the feet of person looking at tv',
 'dog and person are watching television together',
 'person is sitting with their dog watching tv',
 'man relaxing at home watching television with his dog']

In [27]:
## now the cleaned descriptions are ready in "descriptions_test" file.
## Note: No need to add the startseq and endseq tokens as I will remove that from the inference before comparison

### get an idea of the length of GT captions in cleaned descriptions

In [28]:
[gt_cap for desc in descriptions_test.values() for gt_cap in desc][:8]

['black honda motorcycle parked in front of garage',
 'honda motorcycle parked in grass driveway',
 'black honda motorcycle with dark burgundy seat',
 'ma motorcycle parked on the gravel in front of garage',
 'motorcycle with its brake extended standing outside',
 'an office cubicle with four different types of computers',
 'the home office space seems to be very cluttered',
 'an office with desk computer and chair and laptop']

In [29]:
df = pd.DataFrame(data=[gt_cap for desc in descriptions_test.values() for gt_cap in desc], columns=['sent'])
df['caplen'] = df['sent'].str.split().apply(len)
#df['sent'][df['caplen'] == MAX_LENGTH_CAPTION]
df[:8]

,sent,caplen
0,black honda motorcycle parked in front of garage,8
1,honda motorcycle parked in grass driveway,6
2,black honda motorcycle with dark burgundy seat,7
3,ma motorcycle parked on the gravel in front of...,10
4,motorcycle with its brake extended standing ou...,7
5,an office cubicle with four different types of...,9
6,the home office space seems to be very cluttered,9
7,an office with desk computer and chair and laptop,9


In [30]:
## see counts of descriptions with the LONGEST lengths
## we see only one caption has the maximum length
## NOTE: no startseq and endseq tokens present
countdf = df['caplen'].value_counts()
countdf.sort_index(inplace=True, ascending=False)
countdf.head(15)

45     1
42     1
35     1
34     1
33     1
32     2
30     1
29     1
28     2
27     3
26     1
25     6
24     6
23     7
22    11
Name: caplen, dtype: int64

In [31]:
## see counts of descriptions with the SHORTEST lengths
## NOTE: no startseq and endseq tokens present
countdf = df['caplen'].value_counts()
countdf.sort_index(inplace=True, ascending=True)
countdf.head(15)

5      160
6     1971
7     4873
8     6005
9     4864
10    3118
11    1764
12     958
13     512
14     306
15     151
16     117
17      61
18      44
19      26
Name: caplen, dtype: int64

In [32]:
## maximum lenght of description in Test data is 45 without extra tokens, else 47 with startseq and endseq inserted
## But my trained model has max caption length of 49 including the extra tokens for startseq and endseq
df['caplen'].max()

45

In [33]:
## see the description with longest length = 45
df.loc[df['caplen'] == df['caplen'].max()]

,sent,caplen
15996,large square concrete wall which shows people ...,45


In [34]:
del df, countdf

## ALWAYS - Define the RNN Decoder model
## Reload ALL weights from previous training point. Then freeze ONLY the embedding layer as before. Thus, the embedding layer will continue to use the GloVe-200 embeddings matrix weights that were first set during Training Phase 1 setup.


### About Keras Emebedding layer
### 
#### Default initializing of this layer: https://keras.io/api/layers/core_layers/embedding/
######tf.keras.layers.Embedding(
######    input_dim, output_dim,
######    embeddings_initializer="uniform",
######    embeddings_regularizer=None,
######    activity_regularizer=None,
######    embeddings_constraint=None,
######    mask_zero=False,
######    input_length=None,
######    **kwargs
######)

In [35]:
IPDIR_WEIGHTS_IN

'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Weights_In_Run2/'

In [36]:
os.listdir(IPDIR_WEIGHTS_IN)

['Decoder_Run_2_Wt_ep_1.h5',
 'Decoder_Run_2_Wt_ep_10.h5',
 'Decoder_Run_2_Wt_ep_2.h5',
 'Decoder_Run_2_Wt_ep_3.h5',
 'Decoder_Run_2_Wt_ep_4.h5',
 'Decoder_Run_2_Wt_ep_5.h5',
 'Decoder_Run_2_Wt_ep_6.h5',
 'Decoder_Run_2_Wt_ep_7.h5',
 'Decoder_Run_2_Wt_ep_8.h5',
 'Decoder_Run_2_Wt_ep_9.h5']

### IMPORTANT - SPECIFY CORRECT WEIGHTS FILE TO LOAD FROM

In [37]:
RELOAD_WEIGHTS_FILE_NAME = r'Decoder_Run_2_Wt_ep_10.h5'

In [38]:
RELOAD_WEIGHTS_FILE_PATH = IPDIR_WEIGHTS_IN + RELOAD_WEIGHTS_FILE_NAME
print(f"Will reload this file = {RELOAD_WEIGHTS_FILE_PATH}")
print(f"Verfiy the weights in file exists : Check value is True = {os.path.exists(RELOAD_WEIGHTS_FILE_PATH) and os.path.isfile(RELOAD_WEIGHTS_FILE_PATH)}")

Will reload this file = /media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Weights_In_Run2/Decoder_Run_2_Wt_ep_10.h5
Verfiy the weights in file exists : Check value is True = True


In [39]:
def get_model_parameter_counts(_model):
    total_params = _model.count_params()
    total_trainable_params = int(np.sum([np.prod(v.get_shape().as_list()) for v in _model.trainable_variables]))
    total_non_trainable_params = int(np.sum([np.prod(v.get_shape().as_list()) for v in _model.non_trainable_variables]))
    #print(f"Total params = {total_decoder_params}\t Trainable total = {total_decoder_trainable_params}\t Non-trainable total = {total_decoder_non_trainable_params}")
    return total_params, total_trainable_params, total_non_trainable_params

def reload_rnn_decoder_for_inference(_saved_weights_file, _EMBEDDING_DIMS, _VOCAB_SIZE, _MAX_LENGTH_CAPTION):
    if os.path.exists(_saved_weights_file) and os.path.isfile(_saved_weights_file):
        ## Decoder Model defining
        
        ## parameters to define model - sent during function call
        #EMBEDDING_DIMS is initialised earlier while creating embedding matrix
        #VOCAB_SIZE is initialised earlier
        #MAX_LENGTH_CAPTION is initialised earlier
        
        inputs1 = keras.Input(shape=(2048,))
        fe1 = keras.layers.Dropout(0.5)(inputs1)
        fe2 = keras.layers.Dense(256, activation='relu')(fe1)
        
        # partial caption sequence model
        inputs2 = keras.Input(shape=(_MAX_LENGTH_CAPTION,))
        se1 = keras.layers.Embedding(_VOCAB_SIZE, _EMBEDDING_DIMS, mask_zero=True)(inputs2)
        se2 = keras.layers.Dropout(0.5)(se1)
        se3 = keras.layers.LSTM(256)(se2)
        
        # decoder (feed forward) model
        decoder1 = keras.layers.add([fe2, se3])
        decoder2 = keras.layers.Dense(256, activation='relu')(decoder1)
        outputs = keras.layers.Dense(_VOCAB_SIZE, activation='softmax')(decoder2)
        
        # merge the two input models
        reloaded_rnn_decoder_model = keras.models.Model(inputs=[inputs1, inputs2], outputs=outputs)
        
        total_p, trainable_p, non_trainable_p = get_model_parameter_counts(reloaded_rnn_decoder_model)
        print(f"\nCreated RNN Decoder model defined with these paramenters:\nEMBEDDING_DIMS = {_EMBEDDING_DIMS} , VOCAB_SIZE = {_VOCAB_SIZE} , MAX_LENGTH_CAPTION = {_MAX_LENGTH_CAPTION}")
        print(f"\nBEFORE LOADING WEIGHTS AND FREEZING, parameter counts:\nTotal parameters = {total_p} , Trainable parameter = {trainable_p} , Non-trainable parameters = {non_trainable_p}")
        
        
        print(f"\nAttempting to load weights...\n")
        
        ## load the weights
        reloaded_rnn_decoder_model.load_weights(_saved_weights_file)
        print(f"SUCCESS - Reloaded weights from :: {_saved_weights_file}")
        
        ## freeze the embeddings layer weights so they are non-trainable
        reloaded_rnn_decoder_model.trainable = False
        print(f"\nFrozen all layers as this is only for inference and not for training.")
        
        total_p, trainable_p, non_trainable_p = get_model_parameter_counts(reloaded_rnn_decoder_model)
        print(f"\nAFTER FREEZING, parameter counts:\nTotal parameters = {total_p} , Trainable parameter = {trainable_p} , Non-trainable parameters = {non_trainable_p}")
        
        return reloaded_rnn_decoder_model
    else:
        print(f"\nERROR reloading weights. Check weights file exists here = {_saved_weights_file} ;\nOR model setup parameters incompatible with the saved weights file given.")
        return None

In [40]:
print(f"The values to be used in Decoder setup:\nEMBEDDING_DIMS = {EMBEDDING_DIMS} , VOCAB_SIZE = {VOCAB_SIZE} , MAX_LENGTH_CAPTION = {MAX_LENGTH_CAPTION}")
print(f"Weights to be reloaded from here:\n{RELOAD_WEIGHTS_FILE_PATH}")

The values to be used in Decoder setup:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49
Weights to be reloaded from here:
/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Weights_In_Run2/Decoder_Run_2_Wt_ep_10.h5


In [41]:
reloaded_RNN_decoder = reload_rnn_decoder_for_inference(RELOAD_WEIGHTS_FILE_PATH, EMBEDDING_DIMS, VOCAB_SIZE, MAX_LENGTH_CAPTION)
if reloaded_RNN_decoder is None:
    print(f"FATAL ERROR setting up Decoder")
else:
    print(f"\n{type(reloaded_RNN_decoder)}")


Created RNN Decoder model defined with these paramenters:
EMBEDDING_DIMS = 200 , VOCAB_SIZE = 6758 , MAX_LENGTH_CAPTION = 49

BEFORE LOADING WEIGHTS AND FREEZING, parameter counts:
Total parameters = 4146710 , Trainable parameter = 4146710 , Non-trainable parameters = 0

Attempting to load weights...

SUCCESS - Reloaded weights from :: /media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Code/ModelsRuns/ImgCap/SavedData/Thesis_ImgCap_Weights_In_Run2/Decoder_Run_2_Wt_ep_10.h5

Frozen all layers as this is only for inference and not for training.

AFTER FREEZING, parameter counts:
Total parameters = 4146710 , Trainable parameter = 0 , Non-trainable parameters = 4146710

<class 'tensorflow.python.keras.engine.training.Model'>


In [42]:
## Output of command:  model_RNN_decoder.summary() AFTER FREEZING THE EMBEDDINGS WEIGHTS
## NOTE the count of Trainable params has REDUCED
reloaded_RNN_decoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 49)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 49, 200)      1351600     input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [ ]:
## Output of command:  model_RNN_decoder.summary() AFTER FREEZING EVERYTHING
## NOTE the count of Trainable params is ZERO now

#Model: "functional_5"
#__________________________________________________________________________________________________
#Layer (type)                    Output Shape         Param #     Connected to                     
#==================================================================================================
#input_6 (InputLayer)            [(None, 49)]         0                                            
#__________________________________________________________________________________________________
#input_5 (InputLayer)            [(None, 2048)]       0                                            
#__________________________________________________________________________________________________
#embedding_2 (Embedding)         (None, 49, 200)      1351600     input_6[0][0]                    
#__________________________________________________________________________________________________
#dropout_4 (Dropout)             (None, 2048)         0           input_5[0][0]                    
#__________________________________________________________________________________________________
#dropout_5 (Dropout)             (None, 49, 200)      0           embedding_2[0][0]                
#__________________________________________________________________________________________________
#dense_6 (Dense)                 (None, 256)          524544      dropout_4[0][0]                  
#__________________________________________________________________________________________________
#lstm_2 (LSTM)                   (None, 256)          467968      dropout_5[0][0]                  
#__________________________________________________________________________________________________
#add_2 (Add)                     (None, 256)          0           dense_6[0][0]                    
#                                                                 lstm_2[0][0]                     
#__________________________________________________________________________________________________
#dense_7 (Dense)                 (None, 256)          65792       add_2[0][0]                      
#__________________________________________________________________________________________________
#dense_8 (Dense)                 (None, 6758)         1736806     dense_7[0][0]                    
#==================================================================================================
#Total params: 4,146,710
#Trainable params: 0
#Non-trainable params: 4,146,710
#__________________________________________________________________________________________________

## Create the CNN Encoder (Keras Google Inception-v3 pretrained on Imagenet)

In [43]:
    ### Load pre-trained model of Inception-v3 pretrained on Imagenet
    model_inception_v3_pretrained_imagement = tf.keras.applications.InceptionV3(weights='imagenet')
    # Create new model, by removing last layer (output layer) from Inception-V3
    model_CNN_encoder = keras.Model(inputs=model_inception_v3_pretrained_imagement.input, outputs=model_inception_v3_pretrained_imagement.layers[-2].output)
    
    ## should be tensorflow.python.keras.engine.functional.Functional
    type(model_CNN_encoder) 

tensorflow.python.keras.engine.training.Model

# Perform inference on all the 5k images of the TEST DATASET (coco_val2017 data)
### descriptions_test dict has array of cleaned descriptions with image (without jpg) as key

In [44]:
## functions related to using the cnn encoder to extract image features

def preprocess_image_for_Incepv3(_img_path):
    """
    ## Make images suitable for use by Inception-v3 model later
    ##
    ## Resize to (299, 299)
    ## As model needs 4-dim input tensor, add one dimenion to make it (1, 299, 299, 3)
    ## Preprocess the image using custom function of Inception-v3 model
    """
    img = tf.keras.preprocessing.image.load_img(_img_path, target_size=(299, 299))
    #print(f"type={type(img)}") # type(img): type=<class 'PIL.Image.Image'>
    img = tf.keras.preprocessing.image.img_to_array(img) # Converts PIL Image instance to numpy array (299,299,3)
    img = np.expand_dims(img, axis=0) #Add one more dimension: (1, 299, 299, 3) # Inception-V3 requires 4 dimensions
    img = tf.keras.applications.inception_v3.preprocess_input(img) # preprocess image as per Inception-V3 model
    return img  # shape will be (1, 299, 299, 3)

def encode_image(_imgpath, _model_CNN_encoder):
    """
    # Function to encode given image into a vector of size (2048, )
    """
    ## preprocess image per Inception-v3 requirements
    preproc_img = preprocess_image_for_Incepv3(_imgpath)
    ## get encoding vector for image
    encoded_features = _model_CNN_encoder.predict(preproc_img) 
    ## reshape from (1, 2048) to (2048, )
    encoded_features = encoded_features.reshape(encoded_features.shape[1], ) 
    return encoded_features

## functions to actually do the inference

def greedySearch(_decoder_model, _img_encoding, _max_length, _wordtoix = None, _ixtoword = None):
    wordtoix = _wordtoix
    ixtoword = _ixtoword
    in_text = 'startseq'
    for i in range(_max_length):
        sequence = [ wordtoix[w] for w in in_text.split() if w in wordtoix ]
        sequence = keras.preprocessing.sequence.pad_sequences([sequence], maxlen=_max_length)
        yhat = _decoder_model.predict([_img_encoding,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    caption_out = in_text.split()
    #caption_out = caption_out[1:-1]  ## drop the startseq and endseq words at either end
    caption_out = ' '.join(caption_out)
    return caption_out

def do_inference_one_image_totally_new(_infer_image_jpg, _img_encoding, _MAX_LENGTH_CAPTION, _model_RNN_decoder, _wordtoix = None, _ixtoword = None, _DEBUG_SWITCH=False):
    ## show the original image
    image = _img_encoding.reshape((1,2048))
    
    ## briefly show the image about to be inferred if in debug mode
    if _DEBUG_SWITCH:
        plt.ion()
        x = plt.imread(_infer_image_jpg)
        plt.imshow(x)
        plt.show()
        plt.pause(1.5)  ## display for 1.5 seconds
    
    predicted_caption = None
    ## get the prediction caption using greedy search
    predicted_caption = greedySearch(_model_RNN_decoder, image, _MAX_LENGTH_CAPTION, _wordtoix = _wordtoix, _ixtoword = _ixtoword)
    
    if _DEBUG_SWITCH:
        print(f"\nFor image :: {_infer_image_jpg}\n\nInference caption output:\n{ predicted_caption }")
    return predicted_caption

In [45]:
MAX_LENGTH_CAPTION

49

In [46]:
IPDIR_IMGS_COCO_VAL

'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_val2017_5k/val2017/'

In [47]:
imgs_to_infer_full_path_list = [IPDIR_IMGS_COCO_VAL + f for f in os.listdir(IPDIR_IMGS_COCO_VAL)]

In [48]:
## check the key being made to access the descriptions from its dict
for f in imgs_to_infer_full_path_list[:5]:
    print(f"{ os.path.splitext(os.path.basename(f))[0] }")

000000000139
000000000285
000000000632
000000000724
000000000776


In [49]:
## use the encodings and get inferences
comparison_dict = {}  ## key will be the image name, values will be inference caption and the cleaned descriptions
i = 0
total_to_infer = len(imgs_to_infer_full_path_list)
for img_to_infer in imgs_to_infer_full_path_list:
    if i % 50 == 0:
        print(f"Processing {i+1} of {total_to_infer}")
    ## extract just the filename without .jpg extension from the full path
    key4DescDict = os.path.splitext(os.path.basename(img_to_infer))[0]    
    ## Get the encoding by running thru bottleneck
    ## Function to encode given image into a vector of size (2048, )
    img_encoding_for_inference = encode_image(img_to_infer, model_CNN_encoder)
    ## now do the decoder inference using the encoding
    predicted_caption = do_inference_one_image_totally_new(img_to_infer, img_encoding_for_inference, MAX_LENGTH_CAPTION, reloaded_RNN_decoder, wordtoix, ixtoword)
    ## strip off the startseq and endseq - note that startseq is ALWAYS present as the first word, but the endseq MAY NOT ALWAYS be at the end
    predicted_caption = predicted_caption.split(' ')
    if predicted_caption[-1] == 'endseq':
        predicted_caption = ' '.join(predicted_caption[1:-1]) ## remove both ends
    else:
        predicted_caption = ' '.join(predicted_caption[1:])   ## remove only startseq
    ## update the comparison dict
    comparison_dict.update( { key4DescDict : [predicted_caption , descriptions_test[key4DescDict] ] } )
    i += 1

Processing 1 of 5000
Processing 51 of 5000
Processing 101 of 5000
Processing 151 of 5000
Processing 201 of 5000
Processing 251 of 5000
Processing 301 of 5000
Processing 351 of 5000
Processing 401 of 5000
Processing 451 of 5000
Processing 501 of 5000
Processing 551 of 5000
Processing 601 of 5000
Processing 651 of 5000
Processing 701 of 5000
Processing 751 of 5000
Processing 801 of 5000
Processing 851 of 5000
Processing 901 of 5000
Processing 951 of 5000
Processing 1001 of 5000
Processing 1051 of 5000
Processing 1101 of 5000
Processing 1151 of 5000
Processing 1201 of 5000
Processing 1251 of 5000
Processing 1301 of 5000
Processing 1351 of 5000
Processing 1401 of 5000
Processing 1451 of 5000
Processing 1501 of 5000
Processing 1551 of 5000
Processing 1601 of 5000
Processing 1651 of 5000
Processing 1701 of 5000
Processing 1751 of 5000
Processing 1801 of 5000
Processing 1851 of 5000
Processing 1901 of 5000
Processing 1951 of 5000
Processing 2001 of 5000
Processing 2051 of 5000
Processing 2101

## VERIFY WHERE TO PICKLE THE COMPARISON

In [55]:
OPDIR = r'/home/rohit/PyWDUbuntu/thesis/'

In [56]:
OPDIR

'/home/rohit/PyWDUbuntu/thesis/'

In [57]:
COMPARISON_DICT_PICKLE_FILE = r'ImgCap_Run2_TestSet_DescCap_Comparison_WtEp10_1.pkl'

In [58]:
print(f"Picle destination=\n{OPDIR + COMPARISON_DICT_PICKLE_FILE}")

Picle destination=
/home/rohit/PyWDUbuntu/thesis/ImgCap_Run2_TestSet_DescCap_Comparison_WtEp10_1.pkl


In [59]:
## pickle it
with open(OPDIR + COMPARISON_DICT_PICKLE_FILE, 'wb') as handle:
  pickle.dump(comparison_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Success, pickled to:\n{OPDIR + COMPARISON_DICT_PICKLE_FILE}")

Success, pickled to:
/home/rohit/PyWDUbuntu/thesis/ImgCap_Run2_TestSet_DescCap_Comparison_WtEp10_1.pkl


# CALCULATE BLEU SCORE USING NLTK

# SUPER IMPORTANT - CHANGE CONDA ENV FIRST
## DO NOT USE               ce7comb1
## CHANGE TO                ce6idelements1

In [34]:
import nltk.translate.bleu_score as nltk_bleu
import pickle
import pandas as pd

In [35]:
OPDIR = r'/home/rohit/PyWDUbuntu/thesis/'
OPDIR

'/home/rohit/PyWDUbuntu/thesis/'

In [36]:
COMPARISON_DICT_PICKLE_FILE = r'ImgCap_Run2_TestSet_DescCap_Comparison_WtEp10_1.pkl'

In [37]:
print(f"Picle source=\n{OPDIR + COMPARISON_DICT_PICKLE_FILE}")

Picle source=
/home/rohit/PyWDUbuntu/thesis/ImgCap_Run2_TestSet_DescCap_Comparison_WtEp10_1.pkl


## Reload from picled file

In [38]:
with open(OPDIR + COMPARISON_DICT_PICKLE_FILE, 'rb') as handle:
  comparison_dict_reload = pickle.load(handle)
print(f"Reloaded from picle file:\n{OPDIR + COMPARISON_DICT_PICKLE_FILE}")

Reloaded from picle file:
/home/rohit/PyWDUbuntu/thesis/ImgCap_Run2_TestSet_DescCap_Comparison_WtEp10_1.pkl


## Bleu scores with NLTK

### Link: https://ariepratama.github.io/Introduction-to-BLEU-in-python/

In [39]:
## see a few random entries
dict(list(comparison_dict_reload.items())[:15])

{'000000000139': ['living room with couch and television',
  ['woman stands in the dining area at the table',
   'room with chairs table and woman in it',
   'woman standing in kitchen by window',
   'person standing at table in room',
   'living area with television and table']],
 '000000000285': ['brown bear is standing in the grass',
  ['big burly grizzly bear is show with grass in the background',
   'the large brown bear has black nose',
   'closeup of brown bear sitting in grassy area',
   'large bear that is sitting on grass',
   'close up picture of brown bear face']],
 '000000000632': ['bedroom with bed and table with two lamps',
  ['bedroom scene with bookcase blue comforter and window',
   'bedroom with bookshelf full of books',
   'this room has bed with blue sheets and large bookcase',
   'bed and mirror in small room',
   'bed room with neatly made bed window and book shelf']],
 '000000000724': ['stop sign with street sign on it',
  ['stop sign is mounted upside down on i

In [40]:
## see a few random bleu scores
i = 0
for k, v in comparison_dict_reload.items():
    i += 1
    #print('BLEU score: {}'.format(bleu.sentence_bleu(reference, translation)))
    print(f"Caption {i} :: {k + '.jpg'} :: Bleu score = {nltk_bleu.sentence_bleu(v[1], v[0])}")
    if i > 3:
        break

Caption 1 :: 000000000139.jpg :: Bleu score = 0.8241129631093145
Caption 2 :: 000000000285.jpg :: Bleu score = 0.8346850194310856
Caption 3 :: 000000000632.jpg :: Bleu score = 0.6435879088277018
Caption 4 :: 000000000724.jpg :: Bleu score = 0.6314181891184061


In [41]:
## calculate bleu scores for all the 5k images
bscores_all = list()
i = 0
for k, v in comparison_dict_reload.items():
    bscore = nltk_bleu.sentence_bleu(v[1], v[0])
    bscores_all.append([k, v[0], bscore])
    i += 1
print(f"Processed {i} inputs")

/home/rohit/anaconda3/envs/ce6idelements1/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/rohit/anaconda3/envs/ce6idelements1/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Processed 5000 inputs


In [42]:
len(bscores_all)

5000

In [43]:
## see a few random entries
bscores_all[:10]

[['000000000139', 'living room with couch and television', 0.8241129631093145],
 ['000000000285', 'brown bear is standing in the grass', 0.8346850194310856],
 ['000000000632',
  'bedroom with bed and table with two lamps',
  0.6435879088277018],
 ['000000000724', 'stop sign with street sign on it', 0.6314181891184061],
 ['000000000776',
  'teddy bear sitting on top of brown teddy bear',
  0.6395565268169059],
 ['000000000785',
  'man in red jacket and skis on snow covered slope',
  0.7168290718649489],
 ['000000000802',
  'kitchen with refrigerator and refrigerator',
  0.6056679560107702],
 ['000000000872', 'baseball player swinging bat at ball', 0.6205238582310286],
 ['000000000885',
  'man in white shirt playing tennis on tennis court',
  0.7821697863592526],
 ['000000001000',
  'group of people standing around tennis court',
  0.8366930113447216]]

In [45]:
dfbs = pd.DataFrame(bscores_all, columns=['img', 'infcap', 'bsnltk'])
dfbs.sort_values(by=['bsnltk'], inplace = True)

In [46]:
dfbs.head()

,img,infcap,bsnltk
2294,000000484760,clock tower with clock on it,8.580523e-155
2299,000000485130,bed with two beds and two beds,3.831503e-78
819,000000197870,bird perched on the ground next to bird,4.351978e-78
1959,000000015517,train traveling down bridge next to bridge,4.815777e-78
2461,000000439522,man in black jacket and black jacket and black...,5.385075e-02


In [47]:
dfbs.tail()

,img,infcap,bsnltk
1702,000000149770,man riding surfboard on top of wave,1.0
3635,000000199442,man riding wave on top of surfboard,1.0
1663,000000450303,group of people sitting around table with laptops,1.0
611,000000383606,bathroom with sink and mirror,1.0
3581,000000320696,man riding wave on top of surfboard,1.0


In [50]:
bs_max = float(dfbs[['bsnltk']].max(axis=0))
bs_med = float(dfbs[['bsnltk']].median(axis=0))
bs_min = float(dfbs[['bsnltk']].min(axis=0))
bs_avg = float(dfbs[['bsnltk']].mean(axis=0))
bs_std = float(dfbs[['bsnltk']].std(axis=0))
print(f"\t\tBlue Scores:")
print(f"\nMax = {bs_max}\nMedian = {bs_med}\nMin = {bs_min}\nAverage = {bs_avg}\nStd Dev = {bs_std}")

		Blue Scores:

Max = 1.0
Median = 0.6372257075006953
Min = 8.580523318321737e-155
Average = 0.6327242145911766
Std Dev = 0.16428741243707423
